# Text Preprocessing II (week 3)

### WordNet - continuation from week 2
WordNet is a semantically-oriented dictionary of English, similar to a traditional thesaurus but with a richer structure. NLTK includes the English WordNet, with 155,287 words and 117,659 synonym sets. We'll begin by looking at synonyms and how they are accessed in WordNet.
Reference: https://www.nltk.org/book/ch02.html and http://www.nltk.org/howto/wordnet.html

In [1]:
from nltk.corpus import wordnet as wn
wn.synsets('motorcar') 

[Synset('car.n.01')]

In [2]:
# The entity car.n.01 is called a synset, or "synonym set", a collection of synonymous words (or "lemmas")
wn.synset('car.n.01').lemma_names()

['car', 'auto', 'automobile', 'machine', 'motorcar']

In [3]:
wn.synset('car.n.01').definition()

'a motor vehicle with four wheels; usually propelled by an internal combustion engine'

In [4]:
wn.synset('car.n.01').examples()

['he needs a car to get to work']

In [5]:
motorcar = wn.synset('car.n.01')
# hypernyms - parent synsets with broader meaning
types_of_motorcar = motorcar.hypernyms()  
print(types_of_motorcar)

[Synset('motor_vehicle.n.01')]


In [6]:
# from an item to its components (meronyms) 
wn.synset('tree.n.01').part_meronyms() 

[Synset('burl.n.02'),
 Synset('crown.n.07'),
 Synset('limb.n.02'),
 Synset('stump.n.01'),
 Synset('trunk.n.01')]

In [9]:
wn.lemma('supply.n.02.supply').antonyms()

[Lemma('demand.n.02.demand')]

### Week 3
### Word Sense Disambiguation

Lesk Algorithm (http://www.nltk.org/howto/wsd.html; https://www.nltk.org/_modules/nltk/wsd.html) performs the classic Lesk algorithm for Word Sense Disambiguation (WSD) using the definitions of the ambiguous word. Given an ambiguous word and the context in which the word occurs, Lesk returns a Synset with the highest number of overlapping words between the context sentence and different definitions from each Synset.

In [10]:
from nltk.wsd import lesk
sent = ['I', 'went', 'to', 'the', 'bank', 'to', 'deposit', 'money', '.']
print(lesk(sent, 'bank', 'n'))
print(lesk(sent, 'bank'))

Synset('savings_bank.n.02')
Synset('savings_bank.n.02')


In [11]:
# The definitions for "bank" are:
# online version: http://wordnetweb.princeton.edu/perl/webwn
# "bank" in sent is close to the sense of 
# Synset('depository_financial_institution.n.01'). 
from nltk.corpus import wordnet as wn
for ss in wn.synsets('bank'):
    print(ss, ss.definition())

Synset('bank.n.01') sloping land (especially the slope beside a body of water)
Synset('depository_financial_institution.n.01') a financial institution that accepts deposits and channels the money into lending activities
Synset('bank.n.03') a long ridge or pile
Synset('bank.n.04') an arrangement of similar objects in a row or in tiers
Synset('bank.n.05') a supply or stock held in reserve for future use (especially in emergencies)
Synset('bank.n.06') the funds held by a gambling house or the dealer in some gambling games
Synset('bank.n.07') a slope in the turn of a road or track; the outside is higher than the inside in order to reduce the effects of centrifugal force
Synset('savings_bank.n.02') a container (usually with a slot in the top) for keeping money at home
Synset('bank.n.09') a building in which the business of banking transacted
Synset('bank.n.10') a flight maneuver; aircraft tips laterally about its longitudinal axis (especially in turning)
Synset('bank.v.01') tip laterally
Sy

In [12]:
# Test disambiguation of POS tagged "able". - more accurate result
print([(s, s.pos()) for s in wn.synsets('able')])
sent = 'people should be able to marry a person of their choice'.split()
print(lesk(sent, 'able'))
print(lesk(sent, 'able', pos='a'))  # provide a correct synset
# a means ADJECTIVE; s means ADJECTIVE SATELLITE 
# Certain adjectives bind minimal meaning. e.g. "dry", "good", etc. Each of these is the center of an adjective synset in WN.
# Adjective satellites imposes additional commitments on top of the meaning of the central adjective, 
# e.g. "arid" = "dry" + a particular context (i.e. climates)

[(Synset('able.a.01'), 'a'), (Synset('able.s.02'), 's'), (Synset('able.s.03'), 's'), (Synset('able.s.04'), 's')]
Synset('able.s.04')
Synset('able.a.01')


In [13]:
for ss in wn.synsets('able'):
    print(ss, ss.definition())

Synset('able.a.01') (usually followed by `to') having the necessary means or skill or know-how or authority to do something
Synset('able.s.02') have the skills and qualifications to do things well
Synset('able.s.03') having inherent physical or mental ability or capacity
Synset('able.s.04') having a strong healthy body


### Noise Removal
 
Many denoising tasks, such as removing HTML markups, parsing a JSON structure, would need to be implemented prior to tokenization. In our data preprocessing pipeline, we will strip away HTML markup with the help of the BeautifulSoup library.

In [32]:
sample = """<h1>Title Goes Here</h1>

<b>Bolded Text</b>
<i>Italicized Text</i>

<img src="this should all be gone"/>
<a href="this will be gone, too">But this will still be here!</a>

I run. He ran. She is running. Will they stop running?
I talked. She was talking. They talked to them about running. Who ran to the talking runner?

[Some text we don't want to keep is in here]

¡Sebastián, Nicolás, Alejandro and Jéronimo are going to the store tomorrow morning!

something... is! wrong() with.,; this :: sentence.

I can't do this anymore. I didn't know them. Why couldn't you have dinner at the restaurant?

My favorite movie franchises, in order: Indiana Jones; Marvel Cinematic Universe; Star Wars; Back to the Future; Harry Potter.

Don't do it.... Just don't. Billy! I know what you're doing. This is a great little house you've got here.

[This is some other unwanted text]

John: "Well, well, well."
James: "There, there. There, there."

&nbsp;&nbsp;

There are a lot of reasons not to do this. There are 101 reasons not to do it. 1000000 reasons, actually.

I have to go get 2 tutus from 2 different stores, too.

22    45   1067   445

{{Here is some stuff inside of double curly braces.}}

{Here is more stuff in single curly braces.}

[DELETE]

</body>
</html>"""

In [33]:
from bs4 import BeautifulSoup

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

sample = strip_html(sample)
print(sample)

Title Goes Here
Bolded Text
Italicized Text

But this will still be here!

I run. He ran. She is running. Will they stop running?
I talked. She was talking. They talked to them about running. Who ran to the talking runner?

[Some text we don't want to keep is in here]

¡Sebastián, Nicolás, Alejandro and Jéronimo are going to the store tomorrow morning!

something... is! wrong() with.,; this :: sentence.

I can't do this anymore. I didn't know them. Why couldn't you have dinner at the restaurant?

My favorite movie franchises, in order: Indiana Jones; Marvel Cinematic Universe; Star Wars; Back to the Future; Harry Potter.

Don't do it.... Just don't. Billy! I know what you're doing. This is a great little house you've got here.

[This is some other unwanted text]

John: "Well, well, well."
James: "There, there. There, there."

  

There are a lot of reasons not to do this. There are 101 reasons not to do it. 1000000 reasons, actually.

I have to go get 2 tutus from 2 different stores, t

### Expanding contractions
While not mandatory to do at this stage prior to tokenization (you'll find that this statement is the norm for the relatively flexible ordering of text data preprocessing tasks), replacing contractions with their expansions can be beneficial at this point, since our word tokenizer will split words like "didn't" into "did" and "n't." It's not impossible to remedy this tokenization at a later stage, but doing so prior makes it easier and more straightforward.

In [34]:
!pip install contractions
import contractions

text = "I can't go to the movies. We don't want to buy the books."

output = contractions.fix(text)  # e.g., can't -> cannot; don't -> do not
print(output)

I can not go to the movies. We do not want to buy the books.


In [35]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

sample = replace_contractions(sample)  
print(sample)

Title Goes Here
Bolded Text
Italicized Text

But this will still be here!

I run. He ran. She is running. Will they stop running?
I talked. She was talking. They talked to them about running. Who ran to the talking runner?

[Some text we do not want to keep is in here]

¡Sebastián, Nicolás, Alejandro and Jéronimo are going to the store tomorrow morning!

something... is! wrong() with.,; this :: sentence.

I can not do this anymore. I did not know them. Why could not you have dinner at the restaurant?

My favorite movie franchises, in order: Indiana Jones; Marvel Cinematic Universe; Star Wars; Back to the Future; Harry Potter.

do not do it.... Just do not. Billy! I know what you are doing. This is a great little house you have got here.

[This is some other unwanted text]

John: "Well, well, well."
James: "There, there. There, there."

  

There are a lot of reasons not to do this. There are 101 reasons not to do it. 1000000 reasons, actually.

I have to go get 2 tutus from 2 different

### Regular Expression

Many linguistic processing tasks involve pattern matching. For example, we can find words ending with ed using endswith('ed'). Regular expressions give us a more powerful and flexible method for describing the character patterns we are interested in.
Refer to 3.4   Regular Expressions for Detecting Word Patterns (https://www.nltk.org/book/ch03.html)

In [87]:
import re
x = 'groundhog Groundhog Woodchuck woodchuck'
y = re.findall('[gG]roundhog|[Ww]oodchuck',x)
print (y)

['groundhog', 'Groundhog', 'Woodchuck', 'woodchuck']


In [88]:
x = 'My 2 favorite numbers are 19 and 42'
y = re.findall('[0-9]+',x) # [ ] matches only one character, + => one or more
print (y)

['2', '19', '42']


In [89]:
x = '1234567890'
y = re.findall('(456)',x) # ( ) matches all characters as a group
print (y)

['456']


In [90]:
x = 'We just received $10.00 for cookies.'
y = re.findall('$[0-9.]+',x) # $ indicates end of string/line. 
print(y)
y = re.findall('\$[0-9.]+',x) # \ => escape special character $
print (y)

[]
['$10.00']


In [91]:
x = 'Please contact us at: support@abc.com, sales@ABC.com'
y = re.findall('[\w]+@[\w.]+',x) # 
print (y)

['support@abc.com', 'sales@ABC.com']


In [92]:
# \b is a word boundary in regex and backspace in python
y = '\b[a-cA-C]+.com\b'
print(y)
#input to regex is [a-cA-C]+.co => no match
y = re.findall('\b[a-cA-C]+.com\b',x)    
print(y)

# backslash \ => an escape sequence. \\b causes regex input to be \b   
y = re.findall('\\b[a-cA-C]+.com\\b',x) 
print (y)

# r at start of pattern interprets string as raw - treats \b as literal 
y = re.findall(r'\b[a-cA-C]+.com\b',x) 
print (y)


[a-cA-C]+.com
[]
['abc.com', 'ABC.com']
['abc.com', 'ABC.com']


In [93]:
x = 'From: Using the : character'
y = re.findall('^F.+:', x)
print (y)

['From: Using the :']


In [94]:
# ? meant for non-greedy matching - stops at first : match
y = re.findall('^F.+?:', x)  
print (y)

['From:']


### Part of speech tagging (POS)
Part-of-speech tagging aims to assign parts of speech to each word of a given text (such as nouns, verbs, adjectives, and others) based on its definition and its context.

In [14]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\audi7\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\audi7\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\audi7\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [15]:
### POS TAGGING using nltk ###
from nltk.tokenize import word_tokenize

text = word_tokenize("Parts of speech examples: an article, to write, interesting, easily, and, of")
print(nltk.pos_tag(text))

[('Parts', 'NNS'), ('of', 'IN'), ('speech', 'NN'), ('examples', 'NNS'), (':', ':'), ('an', 'DT'), ('article', 'NN'), (',', ','), ('to', 'TO'), ('write', 'VB'), (',', ','), ('interesting', 'VBG'), (',', ','), ('easily', 'RB'), (',', ','), ('and', 'CC'), (',', ','), ('of', 'IN')]


[TextBlob](https://textblob.readthedocs.io/en/dev/) is a Python library for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more.

In [16]:
### POS TAGGING using TextBlob - omit punctuations ### 
input_str="Parts of speech examples: an article, to write, interesting, easily, and, of"
from textblob import TextBlob
result = TextBlob(input_str)
print(result.tags)

[('Parts', 'NNS'), ('of', 'IN'), ('speech', 'NN'), ('examples', 'NNS'), ('an', 'DT'), ('article', 'NN'), ('to', 'TO'), ('write', 'VB'), ('interesting', 'VBG'), ('easily', 'RB'), ('and', 'CC'), ('of', 'IN')]


### Chunking (shallow parsing)
Chunking is a natural language process that identifies constituent parts of sentences (nouns, verbs, adjectives, etc.) and links them to higher order units that have discrete grammatical meanings (**noun groups or phrases, verb groups**, etc.)

In [17]:
# The first step is to determine the part of speech for each word:
input_str="A black television and a white stove were bought for the new apartment of John."
result = TextBlob(input_str)
print(result.tags)

[('A', 'DT'), ('black', 'JJ'), ('television', 'NN'), ('and', 'CC'), ('a', 'DT'), ('white', 'JJ'), ('stove', 'NN'), ('were', 'VBD'), ('bought', 'VBN'), ('for', 'IN'), ('the', 'DT'), ('new', 'JJ'), ('apartment', 'NN'), ('of', 'IN'), ('John', 'NNP')]


In [18]:
# extract noun phrases using TextBlob
# need to install necessary data: $> python -m textblob.download_corpora
result.noun_phrases

WordList(['black television', 'white stove', 'new apartment', 'john'])

In [20]:
# extract noun phrases using NLTK
# refer to "7. Extracting Information from Text" (https://www.nltk.org/book/ch07.html)
reg_exp = "NP: {<DT>?<JJ>*<NN>}"   # need to define gramma (det, adj, nn) before parsing a sentence.
rp = nltk.RegexpParser(reg_exp)
result1 = rp.parse(result.tags) #parse words with POS
print(result1)

(S
  (NP A/DT black/JJ television/NN)
  and/CC
  (NP a/DT white/JJ stove/NN)
  were/VBD
  bought/VBN
  for/IN
  (NP the/DT new/JJ apartment/NN)
  of/IN
  John/NNP)


In [21]:
result1.draw() # result on pop-up window

In [22]:
# Define grammar before parsing
grammar = r"""
  NP: {<DT|JJ|NN.*>+}          # Chunk sequences of DT, JJ, NN
  PP: {<IN><NP>}               # Chunk prepositions followed by NP
  VP: {<VB.*><NP|PP|CLAUSE>+$} # Chunk verbs and their arguments
  CLAUSE: {<NP><VP>}           # Chunk NP, VP
  """
rp = nltk.RegexpParser(grammar)
result2 = rp.parse(result.tags)
print(result2)

(S
  (NP A/DT black/JJ television/NN)
  and/CC
  (NP a/DT white/JJ stove/NN)
  were/VBD
  (VP
    bought/VBN
    (PP for/IN (NP the/DT new/JJ apartment/NN))
    (PP of/IN (NP John/NNP))))


In [23]:
result2.draw() # result on pop-up window

### n-grams
The TextBlob.ngrams() method returns a list of tuples of n successive words.
[TextBlob](https://textblob.readthedocs.io/en/dev/) is a Python library for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more.

In [95]:
# use TextBlob
!pip install TextBlob
from textblob import TextBlob
blob = TextBlob("Now is better than never.")
blob.ngrams(n=2)

[WordList(['Now', 'is']),
 WordList(['is', 'better']),
 WordList(['better', 'than']),
 WordList(['than', 'never'])]

In [96]:
blob.ngrams(n=3)

[WordList(['Now', 'is', 'better']),
 WordList(['is', 'better', 'than']),
 WordList(['better', 'than', 'never'])]

In [97]:
# use nltk
from nltk.util import ngrams, bigrams
input_str = "NLTK is a leading platform for building Python programs to work with human language data."
tokens = word_tokenize(input_str)
list(ngrams(tokens, 2))

[('NLTK', 'is'),
 ('is', 'a'),
 ('a', 'leading'),
 ('leading', 'platform'),
 ('platform', 'for'),
 ('for', 'building'),
 ('building', 'Python'),
 ('Python', 'programs'),
 ('programs', 'to'),
 ('to', 'work'),
 ('work', 'with'),
 ('with', 'human'),
 ('human', 'language'),
 ('language', 'data'),
 ('data', '.')]

In [98]:
list(bigrams(tokens))

[('NLTK', 'is'),
 ('is', 'a'),
 ('a', 'leading'),
 ('leading', 'platform'),
 ('platform', 'for'),
 ('for', 'building'),
 ('building', 'Python'),
 ('Python', 'programs'),
 ('programs', 'to'),
 ('to', 'work'),
 ('work', 'with'),
 ('with', 'human'),
 ('human', 'language'),
 ('language', 'data'),
 ('data', '.')]

In [99]:
list(ngrams(tokens, 3))

[('NLTK', 'is', 'a'),
 ('is', 'a', 'leading'),
 ('a', 'leading', 'platform'),
 ('leading', 'platform', 'for'),
 ('platform', 'for', 'building'),
 ('for', 'building', 'Python'),
 ('building', 'Python', 'programs'),
 ('Python', 'programs', 'to'),
 ('programs', 'to', 'work'),
 ('to', 'work', 'with'),
 ('work', 'with', 'human'),
 ('with', 'human', 'language'),
 ('human', 'language', 'data'),
 ('language', 'data', '.')]

### Find most common ngrams

In [101]:
from collections import Counter
from nltk import ngrams
bigtxt = open('big.txt').read()
ngram_counts = Counter(ngrams(bigtxt.split(), 3)) #split text to tokens
ngram_counts.most_common(10)

[(('one', 'of', 'the'), 332),
 (('out', 'of', 'the'), 244),
 (('of', 'the', 'United'), 235),
 (('that', 'he', 'was'), 191),
 (('the', 'United', 'States'), 184),
 (('that', 'it', 'was'), 180),
 (('and', 'in', 'the'), 174),
 (('met', 'with', 'in'), 173),
 (('up', 'to', 'the'), 159),
 (('part', 'of', 'the'), 158)]

In [102]:
bigtxt = open('mbox.txt').read()
ngram_counts = Counter(ngrams(bigtxt.split(), 2))
ngram_counts.most_common(10)

[(('Received:', 'from'), 12579),
 (('with', 'ESMTP'), 7188),
 (('ESMTP', 'id'), 7188),
 (('Dec', '2007'), 7063),
 (('Nov', '2007'), 6810),
 (('-0500', 'Received:'), 5843),
 (('for', '<source@collab.sakaiproject.org>;'), 5391),
 (('text/plain;', 'charset=UTF-8'), 5391),
 (('+0000', '(GMT)'), 4932),
 (('from', 'murder'), 3594)]

### BOW vector generation

### Count Vectors as features
Count Vector (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) is a matrix notation of the dataset in which every row represents a document from the corpus, every column represents a term from the corpus, and every cell represents the frequency count of a particular term in a particular document.

In [103]:
### Create feature vectors using CountVectorizer 
### binary parameter used to indicate word presence

from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'UNC played Duke in basketball',
    'Duke lost the basketball game',
    'I ate a sandwich'
]
vectorizer = CountVectorizer(binary=True) # indicate presence/non-presence
# Return a dense matrix representation of this CSR (Compressed Sparse Row) matrix.
# https://machinelearningmastery.com/sparse-matrices-for-machine-learning/ 
X = vectorizer.fit_transform(corpus).todense() 

print(vectorizer.vocabulary_) # Print the list of words in the vocabulary
for i, document in enumerate(corpus):  # Shows condensed version of the feature vectors
    print(document, '=', X[i])

{'unc': 9, 'played': 6, 'duke': 2, 'in': 4, 'basketball': 1, 'lost': 5, 'the': 8, 'game': 3, 'ate': 0, 'sandwich': 7}
UNC played Duke in basketball = [[0 1 1 0 1 0 1 0 0 1]]
Duke lost the basketball game = [[0 1 1 1 0 1 0 0 1 0]]
I ate a sandwich = [[1 0 0 0 0 0 0 1 0 0]]


In [14]:
### CountVectorizer - Creating feature vectors with frequencies of words

corpus = ['The dog ate a sandwich, the wizard transfigured a sandwich, \
and I ate a sandwich']
vectorizer = CountVectorizer(stop_words='english')

### Print the class model with it's parameters
print(vectorizer.fit(corpus), '\n')

### Print the list of words in the vocabulary
print(vectorizer.vocabulary_, '\n')

### Shows the vector with transformed numerical values (word frequency)
print(vectorizer.transform(corpus), '\n')

### Shows condensed version of the feature vectors
print(vectorizer.transform(corpus).todense())

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None) 

{'dog': 1, 'ate': 0, 'sandwich': 2, 'wizard': 4, 'transfigured': 3} 

  (0, 0)	2
  (0, 1)	1
  (0, 2)	3
  (0, 3)	1
  (0, 4)	1 

[[2 1 3 1 1]]


### TF-IDF Vectors as features

TF-IDF Vectors (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) can be generated.

In [105]:
### Creating TF and TF-IDF feature vectors using TfidfTransformer. 
### Note the change in IDF feature values with different number of documents in corpus

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
corpus = [
    'The dog ate a sandwich and I ate a sandwich',
    'The wizard transfigured a sandwich'
]
vectorizer = CountVectorizer(stop_words='english')
# None for no normalization.
transformer = TfidfTransformer(use_idf=False, norm=None) # to get TFIDF values  
transformerIDF = TfidfTransformer(use_idf=True, norm=None) # to get IDF values
X = vectorizer.fit_transform(corpus)
print('Vocabulary:\n', vectorizer.vocabulary_)
print('Count vectors:\n', X.todense())
print('TF vectors:\n', transformer.fit_transform(X).todense())  # divided by the document length
print('TF-IDF vectors:\n', transformerIDF.fit_transform(X).todense())

Vocabulary:
 {'dog': 1, 'ate': 0, 'sandwich': 2, 'wizard': 4, 'transfigured': 3}
Count vectors:
 [[2 1 2 0 0]
 [0 0 1 1 1]]
TF vectors:
 [[2. 1. 2. 0. 0.]
 [0. 0. 1. 1. 1.]]
TF-IDF vectors:
 [[2.81093022 1.40546511 2.         0.         0.        ]
 [0.         0.         1.         1.40546511 1.40546511]]


In [106]:
### TfidfVectorizer - combine CountVectorizer and TfidfTransformer

from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'The dog ate a sandwich and I ate a sandwich',
    'The wizard transfigured a sandwich'
]
vectorizer = TfidfVectorizer(stop_words='english', norm=None)
print(vectorizer.fit_transform(corpus).todense(), "\n")

[[2.81093022 1.40546511 2.         0.         0.        ]
 [0.         0.         1.         1.40546511 1.40546511]] 



In [108]:
idf = vectorizer.idf_
print (dict(zip(vectorizer.get_feature_names(), idf)))

# formula : E.g., "ate" in the first column of the first document
# df = 1 -- "ate" appeared only in one document
# N = 2 -- total number of documents
# idf = ln((N + 1)/(df + 1)) + 1 = log(3/2) + 1 = 1.4054651081081644

# tf = 2
# tfidf = tf * idf = 2 * 1.4054651081081644 = 2.81093022

{'ate': 1.4054651081081644, 'dog': 1.4054651081081644, 'sandwich': 1.0, 'transfigured': 1.4054651081081644, 'wizard': 1.4054651081081644}


In [110]:
### Norm='l2' used to normalize term vectors.
### Creating TF and TF-IDF feature vectors using TfidfTransformer. 
### Note the change in IDF feature values with different number of documents in corpus

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
corpus = [
    'The dog ate a sandwich and I ate a sandwich',
    'The wizard transfigured a sandwich'
]
vectorizer = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(use_idf=False)     # Norm='l2' used to normalize term vectors.
transformerIDF = TfidfTransformer(use_idf=True)
X = vectorizer.fit_transform(corpus)
print('Vocabulary:\n', vectorizer.vocabulary_)
print('Count vectors:\n', X.todense())
print('TF vectors:\n', transformer.fit_transform(X).todense())  # divided by the document length
print('TF-IDF vectors:\n', transformerIDF.fit_transform(X).todense())

Vocabulary:
 {'dog': 1, 'ate': 0, 'sandwich': 2, 'wizard': 4, 'transfigured': 3}
Count vectors:
 [[2 1 2 0 0]
 [0 0 1 1 1]]
TF vectors:
 [[0.66666667 0.33333333 0.66666667 0.         0.        ]
 [0.         0.         0.57735027 0.57735027 0.57735027]]
TF-IDF vectors:
 [[0.75458397 0.37729199 0.53689271 0.         0.        ]
 [0.         0.         0.44943642 0.6316672  0.6316672 ]]


In [111]:
### Norm='l2' used to normalize term vectors.
### TfidfVectorizer - combine CountVectorizer and TfidfTransformer

from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'The dog ate a sandwich and I ate a sandwich',
    'The wizard transfigured a sandwich'
]
vectorizer = TfidfVectorizer(stop_words='english')      # Norm='l2' used to normalize term vectors.
print(vectorizer.fit_transform(corpus).todense(), "\n")

[[0.75458397 0.37729199 0.53689271 0.         0.        ]
 [0.         0.         0.44943642 0.6316672  0.6316672 ]] 

